#Intentamos analizar de forma previa los datos

In [1]:
import pandas as pd
import matplotlib as plot
import nltk 

In [2]:
train=pd.read_table('../inputs/train.tsv')
train['len']=train['item_description'].str.len()
train['cant']=1

In [3]:
train['item_description_upper']= train['item_description'].str.upper()
txtu=train['item_description_upper'].str.cat(sep='.')
txt=train['item_description'].str.cat(sep='.')
len(txtu),len(train['item_description_upper'])

(217504739, 1482535)

In [8]:
txtu[0:300]

'NO DESCRIPTION YET.THIS KEYBOARD IS IN GREAT CONDITION AND WORKS LIKE IT CAME OUT OF THE BOX. ALL OF THE PORTS ARE TESTED AND WORK PERFECTLY. THE LIGHTS ARE CUSTOMIZABLE VIA THE RAZER SYNAPSE APP ON YOUR PC..ADORABLE TOP WITH A HINT OF LACE AND A KEY HOLE IN THE BACK! THE PALE PINK IS A 1X, AND I AL'

In [9]:
#sent=nltk.sent_tokenize(txt)
wordu=nltk.word_tokenize(txtu)
wordu

['NO',
 'DESCRIPTION',
 'YET.THIS',
 'KEYBOARD',
 'IS',
 'IN',
 'GREAT',
 'CONDITION',
 'AND',
 'WORKS',
 'LIKE',
 'IT',
 'CAME',
 'OUT',
 'OF',
 'THE',
 'BOX',
 '.',
 'ALL',
 'OF',
 'THE',
 'PORTS',
 'ARE',
 'TESTED',
 'AND',
 'WORK',
 'PERFECTLY',
 '.',
 'THE',
 'LIGHTS',
 'ARE',
 'CUSTOMIZABLE',
 'VIA',
 'THE',
 'RAZER',
 'SYNAPSE',
 'APP',
 'ON',
 'YOUR',
 'PC..ADORABLE',
 'TOP',
 'WITH',
 'A',
 'HINT',
 'OF',
 'LACE',
 'AND',
 'A',
 'KEY',
 'HOLE',
 'IN',
 'THE',
 'BACK',
 '!',
 'THE',
 'PALE',
 'PINK',
 'IS',
 'A',
 '1X',
 ',',
 'AND',
 'I',
 'ALSO',
 'HAVE',
 'A',
 '3X',
 'AVAILABLE',
 'IN',
 'WHITE',
 '!',
 '.NEW',
 'WITH',
 'TAGS',
 '.',
 'LEATHER',
 'HORSES',
 '.',
 'RETAIL',
 'FOR',
 '[',
 'RM',
 ']',
 'EACH',
 '.',
 'STAND',
 'ABOUT',
 'A',
 'FOOT',
 'HIGH',
 '.',
 'THEY',
 'ARE',
 'BEING',
 'SOLD',
 'AS',
 'A',
 'PAIR',
 '.',
 'ANY',
 'QUESTIONS',
 'PLEASE',
 'ASK',
 '.',
 'FREE',
 'SHIPPING',
 '.',
 'JUST',
 'GOT',
 'OUT',
 'OF',
 'STORAGE.COMPLETE',
 'WITH',
 'CERTIFICAT

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data specifiying a minimum document frequency of 5
vect = TfidfVectorizer(min_df=5).fit(wordu)
len(vect.get_feature_names())

46209

In [14]:
import gensim
id_map = dict((v, k) for k, v in vect.vocabulary_.items())
X = vect.fit_transform(wordu)
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
ldamodel = gensim.models.ldamodel.LdaModel(corpus, id2word=id_map, num_topics=10, passes=25, random_state=34)
ldamodel.print_topics(num_topics=10, num_words=10)

KeyboardInterrupt: 

In [ ]:
len(set(sent))*100/len(sent)

In [ ]:
stemmer=nltk.stem.PorterStemmer()
swordu=[stemmer.stem(word) for word in wordu]

In [ ]:
len(sent),len(wordu),len(set(swordu))

In [ ]:
fdist_wordu = nltk.FreqDist(swordu)
fdist_sent = nltk.FreqDist(sent)

In [ ]:
len(fdist_sent),len(fdist_wordu)

In [ ]:
%matplotlib inline
fdist_sent.plot(100)

In [ ]:
fdist_sent.most_common(500)

In [ ]:
fdist_wordu.most_common(50)

In [ ]:
tot_sent=0
tot_sent_rep=0
for word, freq in fdist_sent.most_common(len(fdist_sent)):
    tot_sent+=freq
    if freq > 1:
        tot_sent_rep+=freq
print(tot_sent,tot_sent_rep)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

corpus = [ 'This is a sentence',
           'Another sentence is here',
           'Wait for another sentence',
           'The sentence is coming',
           'The sentence has come'
         ]

x = vectorizer.fit_transform(corpus)
print(pd.DataFrame(x.A, columns=vectorizer.get_feature_names()).to_string())